In [26]:
# Data Extraction from letour site. 
# This module will download raw data about every rider of tour de france for every year since 1913. We will use required packages for html parsing & data will be postprocessed and moved to CSV format for our project use. 

In [27]:
#Needed Packages
import numpy as np
import pandas as pd
import requests # for handling http requests to get resposne 
from bs4 import BeautifulSoup #for parsing

import os  # OS dependent modules wil be imported depending on which os we're on.
import subprocess #  start new applications right from the Python (running scripts from git etc.,)
import collections #to store collections of data
import re  # to support regular expression through libraries
from pathlib import Path
import datetime 
import time

In [28]:
# Generate the urls that we need to download the raw HTML pages from original letour.fr website  (https://www.letour.fr/en/history)
#Variables used to generate URL -  prefixurl /Historypage for each year data
prefixurl = 'http://www.letour.fr'
HISTORYPAGE = 'https://www.letour.fr/en/history'

In [29]:
# Headers
headers = {'Accept': 'text/html', 'User-Agent': 'python-requests/1.2.0','Accept-Charset': 'utf-8','accept-encoding': 'deflate, br'}


In [30]:
# History link w/header
resulthistpage = requests.get(HISTORYPAGE, allow_redirects=True, headers=headers)
souphistory = BeautifulSoup(resulthistpage.text, 'html.parser')

In [31]:
# Look for select tag for history links
select_tag_histo = souphistory.find_all("button",{"class" :"dateTabs__link"})

LH = [x["data-tabs-ajax"] for x in select_tag_histo]

dflink = pd.DataFrame({'TDFHistorylink':LH})

### Function for time

In [32]:
#Function that will convert HH MM SS to seconds
def calcTotalSeconds(row,mode):

   val = sum(x * int(t) for x, t in zip([3600, 60, 1], row.replace("h",":").replace("'",":").replace('"',':').replace(" ","").replace("+","").replace("-","0").split(":")))
   
   if ((mode=='Gap') and val > 180000) :
        val=0
    
   return val

### Function to retrieve elements from a source HTML page located on an input url

In [33]:
def getstagesNrank(i_url):
    resultfull = requests.get(i_url, allow_redirects=True)
    result = resultfull.text
    resultstatus = resultfull.status_code
    
    print(i_url + ' ==> HTTP STATUS = ' + str(resultstatus))
    
    soup = BeautifulSoup(result, 'html.parser')  
    h=soup.find('h3')
    year=int(h.text[-4:])

    # Find select tag
    select_tag = soup.find("select")

    # find all option tag inside select tag
    options = select_tag.find_all("option")

    cols = ['Year','TotalTDFDistance','Stage']
    lst = []
    
    #search for stages
    distance=soup.select("[class~=statsInfos__number]")[1].contents
               
    #search for distance of the TDF edition
    for option in options:
       lst.append([year,int(distance[0].replace(" ","")),option.text])
               
    dfstages = pd.DataFrame(lst, columns=cols)
 
    
    # Find select tag for ranking racers
    rankingTable = soup.find("table")

    dfrank = pd.read_html(str(rankingTable))[0]

    dfrank['Year']=year
    dfrank['Distance (km)']=int(distance[0].replace(" ",""))
    dfrank['Number of stages']=len(dfstages)
    dfrank['TotalSeconds']=dfrank['Times'].apply(lambda x:calcTotalSeconds(x,'Total'))
    dfrank['GapSeconds']=dfrank['Gap'].apply(lambda x:calcTotalSeconds(x,'Gap'))
    
   #-----------
    # Split up time
    dfrank = dfrank.reset_index()
    dfrank["Times"].apply(lambda x: re.sub("[^0-9]", " ", str(x)).split("  "))
    dfrank["Hours"] = np.NaN
    dfrank["Minutes"] = np.NaN
    dfrank["Seconds"] = np.NaN
    for i in range(len(dfrank)):
        x = dfrank.loc[i, "Times"]
        z = re.sub("[^0-9]", " ", str(x)).split("  ")
        if len(z) == 4:
            z = [int(zi) for zi in z[:3]]
        if len(z) < 3:
            z = [np.NaN, np.NaN, np.NaN]
        dfrank.loc[i, "Hours"] = z[0]
        dfrank.loc[i, "Minutes"] = z[1]
        dfrank.loc[i, "Seconds"] = z[2]
        dfrank["TotalSeconds"] = dfrank["Hours"] * 3600 + dfrank["Minutes"] * 60 + dfrank["Seconds"]
    #-----------
    
    return dfstages, dfrank

### Loop on the dflink dataframe to get data from each url source

In [34]:
dfstagesres=[]
dfstagesrestmp=[]
dfrankres=[]
dfrankrestmp=[]
dfrankoutput=[]

for index, row  in dflink.iterrows():
    url = prefixurl+row['TDFHistorylink']
    try :
      if index >= 12 :  # limit from 1919 (data need to be cleaned a little bit more before that)
        dfstagesres, dfrankres=getstagesNrank(url)
        dfstagesrestmp= dfstagesres.append(dfstagesrestmp, ignore_index=True)   # do not use index labels if true
        dfrankrestmp=dfrankres.append(dfrankrestmp, ignore_index=True) # do not use index labels if true
        
        
    except:
      raise

    
dfstageoutput=dfstagesrestmp
dfrankoutput=dfrankrestmp

http://www.letour.fr/en/block/history/10719/f15cf81b7599ea113d2b1d614f73bf83 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10720/53ac40095834ae1aae5d197aa730799a ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10721/c663ba39ac60d58c55a1a787a23025fc ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10722/cc5d0ba48a5d9bd9ee5e222160452b6f ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10723/9212137527c3f19c0cfccd1d07d80649 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10724/61621b1b7012e711ae7457a04e123aab ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10725/326ccc005b547ad2ad33e2fc03d27159 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10726/417b583aab876fdb00496da5c2be1a7d ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10727/0ab2dd4f425fcdcb9cc0a1511f49c494 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10728/58abb1c5f80d5d1b088ead3c32a5a815 ==> HTTP STATUS = 200
http://www

http://www.letour.fr/en/block/history/10802/814bff61212e58cf7c2beaddb1c083b3 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10803/757e8e81b7183fde7329fd8ab9b69783 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10804/89b36b01ffe439e016ec1d59c57b63d1 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10805/1ec24b4375d16b922c9c9c489d25465c ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10806/196e36cfe7aff4b5fc2d1055f7dcf864 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10807/d538a7fbdfc0d657fbf064561840fbb1 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10808/3f166bedb535ee9bff2fb7ead0a7812c ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10809/7be3a459d846b4672915c576cb7ed6b9 ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/10810/7035a5dc53631209d3581f64a433b10d ==> HTTP STATUS = 200
http://www.letour.fr/en/block/history/11818/f34c3404d95a697dcf77d4cd8e8278fa ==> HTTP STATUS = 200
http://www

In [36]:
dfrankoutput.to_csv("riders.csv")

### Clean up the Data

In [37]:
# Fix result types
dfrankoutput["ResultType"] = "time"
dfrankoutput.loc[dfrankoutput["Year"].isin([1905,1906,1908]),"ResultType"] = "null"
dfrankoutput.loc[dfrankoutput["Year"].isin([1907,1909,1910,1911,1912]),"ResultType"] = "points"

In [38]:
# Fix this weird bug for e.g. year 2006 and 1997
for year in np.unique(dfrankoutput["Year"]):
    tmp = dfrankoutput[dfrankoutput["Year"]==year].reset_index()
    if tmp.loc[0]["TotalSeconds"] > tmp.loc[2]["TotalSeconds"]:
        print(year)
# Okay seems to be only for 2006 and 1997

1997
2006


In [39]:
tmp = dfrankoutput[dfrankoutput["Year"]==2006].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==2006,"TotalSeconds"] = ts

tmp = dfrankoutput[dfrankoutput["Year"]==1997].reset_index()
ts = np.array(tmp["TotalSeconds"])
gs = np.array(tmp["GapSeconds"])
ts[1:] = ts[0]+gs[1:]

dfrankoutput.loc[dfrankoutput["Year"]==1997,"TotalSeconds"] = ts

In [40]:
dfrankoutput.to_csv("TDF_Riders_History.csv")
dfstageoutput.to_csv("TDF_Stages_History.csv")